In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import deque
import copy
import time

In [30]:
import numpy as np


# all array inputs must be of the type np.array

class my_env:
    #def __init__(self, uav_number, user_number, size, user_loc, noise, height, transmit_pow, safe_dist): # initializes environment
    def __init__(self):
        self.uav_number = 2
        self.user_number = 5
        self.size = np.array([10,10])
        self.user_loc = np.array([[-5,-5,5,5,0],[-5,5,-5,5,0]])
        self.safe_dist = 5
        self.noise = 0.01
        self.height = 7
        self.transmit_pow = 1
        self.observation_space = 2*self.uav_number + self.user_number

        ### Parameters
        self.actions = 5**(self.uav_number) # 0: 10 units Left , 1: 10 units up , 2: 10 units right, 3: 10 units down, 4: stay
        self.user_profile = 1*np.ones(self.user_number)
        ###

    def my_reward(self, uav_loc, scheduling, user_prof): # calculate reward
        num_agents = np.shape(uav_loc)[1]
        num_user = np.shape(self.user_loc)[1]

        pref = np.zeros((num_agents,num_user))
        pref_rate = np.zeros((num_agents,num_user))
        for i in range(np.shape(pref)[0]):
            for j in range(np.shape(pref)[1]):
                interf = 0
                for k in range(np.shape(pref)[0]):
                    if k != i:
                        interf = interf + self.transmit_pow/(self.height**2 + np.linalg.norm(uav_loc[:,k]-self.user_loc[:,j]))
                pref[i,j] = user_prof[j]*np.log2(1 + self.transmit_pow/(self.height**2 + np.linalg.norm(uav_loc[:,i]-self.user_loc[:,j]))/(interf+self.noise))
                #pref[i,j] = np.log2(1 + self.transmit_pow/(self.height**2 + np.linalg.norm(uav_loc[:,i]-self.user_loc[:,j]))/(interf+self.noise))


        ####### ---- penalty
        penalty = 0

        for i in range(np.shape(pref)[0]):
            for k in range(np.shape(pref)[0]):
                if k != i:
                    penalty = penalty + (np.linalg.norm(uav_loc[:,k]-uav_loc[:,i]) < self.safe_dist)

        # reward = np.sum(np.multiply(scheduling,pref)) - np.log2(1 + self.transmit_pow/(self.height**2)/self.noise )*penalty/2
        reward = np.sum(np.multiply(scheduling,pref)) - 1000*penalty/2

        if penalty != 0:
            done =  True
        else:
            done = False

        return reward , done


    def next_state(self, action1, uav_loc, user_prof, scheduling):
        if action1<5:
            action = np.base_repr(action1,base=5,padding = 1)
        else :
            action = np.base_repr(action1,base=5,padding = 0)

        for i in range(len(action)):
            if action[i] == "0":

                uav_loc[:,i] = uav_loc[:,i] + (-2*np.array([1,0]))
                uav_loc[0,i] = uav_loc[0,i] + (uav_loc[0,i]<-self.size[0]/2)*(-self.size[0]/2-uav_loc[0,i])

            elif action[i] == "1":

                uav_loc[:,i] = uav_loc[:,i] + (2*np.array([0,1]))
                uav_loc[1,i] = uav_loc[1,i] + (uav_loc[1,i]>self.size[1]/2)*(self.size[1]/2-uav_loc[1,i])

            elif action[i] == "2":

                uav_loc[:,i] = uav_loc[:,i] + (2*np.array([1,0]))
                uav_loc[0,i] = uav_loc[0,i] + (uav_loc[0,i]>self.size[0]/2)*(self.size[0]/2-uav_loc[0,i])

            elif action[i] == "3":

                uav_loc[:,i] = uav_loc[:,i] + (-2*np.array([0,1]))
                uav_loc[1,i] = uav_loc[1,i] + (uav_loc[1,i]<-self.size[1]/2)*(-self.size[1]/2-uav_loc[1,i])

            else :
                pass

        #user_prof = user_prof + (0.1 * ~(np.sum(scheduling,axis=0)>0))
        # user_prof = user_prof - (np.min(user_prof)>5)*5
        user_prof = 1/(0.9/user_prof + 0.1*np.sum(scheduling,axis=0))

        return np.hstack((uav_loc.flatten() , user_prof))

    def init_state(self):

        while True:
            a = np.random.uniform(-self.size[0]/2,self.size[0]/2,size = (1,self.uav_number))
            b = np.random.uniform(-self.size[1]/2,self.size[1]/2,size = (1,self.uav_number))
            c = np.vstack((a,b))
            if np.linalg.norm(c[:,0]-c[:,1])>self.safe_dist:
                break
        
 

        return np.hstack((a.flatten(),b.flatten(),self.user_profile))



import cvxpy as cp
import numpy as np


def schedule(user_prof,user_loc,UAV_loc,transmit_pow,noise,height):


    num_agents = np.shape(UAV_loc)[1]
    num_user = np.shape(user_loc)[1]

    A = cp.Variable((num_agents,num_user))

    pref = np.zeros((num_agents,num_user))
    for i in range(np.shape(pref)[0]):
        for j in range(np.shape(pref)[1]):
            interf = 0
            for k in range(np.shape(pref)[0]):
                if k != i:
                    interf = interf + transmit_pow/(height**2 + np.linalg.norm(UAV_loc[:,k]-user_loc[:,j]))
            pref[i,j] = user_prof[j]*np.log2(1 + transmit_pow/(height**2 + np.linalg.norm(UAV_loc[:,i]-user_loc[:,j]))/(interf+noise))

    objective = cp.Minimize(-1*cp.sum(cp.multiply(A,pref)))

    constraints = [A>=0,A<=1]
    constraints.append(cp.sum(A,axis = 0) <= 1)
    constraints.append(cp.sum(A,axis = 1) <= 1)

    problem = cp.Problem(objective, constraints)

    problem.solve()

    return np.round(A.value)

In [31]:


# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#device = "cpu"

Using device: cuda


In [32]:
# Hyperparameters
GAMMA = 0.99
LR = 0.001
EPISODES = 10000
STEPS_PER_EPISODE = 200

QUAD = my_env()

def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)

# # Define Actor network
# class Actor(nn.Module):
#     def __init__(self, state_dim, action_dim):
#         super(Actor, self).__init__()
#         self.fc1 = nn.Linear(state_dim, 1024)
#         self.fc2 = nn.Linear(1024, 1024)
#         self.fc3 = nn.Linear(1024, 1024)  # Single output for continuous action
#         self.fc4 = nn.Linear(1024, action_dim)
#         self.softmax = nn.Softmax(dim = -1)

#     def forward(self, state):
#         x = torch.relu(self.fc1(state))
#         x = torch.relu(self.fc2(x))
#         x = torch.relu(self.fc3(x))
#         #return self.softmax(F.sigmoid(self.fc4(x)))  # Output probability distribution
#         return F.softmax(self.fc4(x), dim = -1)
#         #return F.softmax(self.fc4(x), dim=-1)

#Define Actor network
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 256)
        self.fc2 = nn.Linear(256, action_dim)
        self.softmax = nn.Softmax(dim = -1)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        return (F.softmax(self.fc2(x),dim = -1))




# Define Critic network
class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, 1)  # Output a single value

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return self.fc4(x)  # State value estimation



In [33]:

# Initialize networks on GPU
actor = Actor(QUAD.observation_space,QUAD.actions).to(device)
critic = Critic(QUAD.observation_space).to(device)

# for param in actor.parameters():
#     if param.ndimension() > 1:  # Check if it's a weight (not a bias)
#         torch.nn.init.constant_(param, 0.0001)

actor_prev = Actor(QUAD.observation_space,QUAD.actions).to(device)
critic_prev = Critic(QUAD.observation_space).to(device)


actor.apply(weight_init)
critic.apply(weight_init)

# Optimizers
actor_optimizer = optim.Adam(actor.parameters(), lr=LR)
critic_optimizer = optim.Adam(critic.parameters(), lr=LR)



In [34]:
def weight_change(model1,model2):
    weight_diff = 0
    for param1, param2 in zip(model1.parameters(), model2.parameters()):
        weight_diff += torch.sum(torch.abs(param1 - param2)).item()
    return weight_diff

In [35]:
tic = time.time()

reward_acul = np.empty(0)

actor_prev.load_state_dict(actor.state_dict())
critic_prev.load_state_dict(critic.state_dict())


for episode in range(1,EPISODES+1):
    state = QUAD.init_state()  # Get initial state
    total_reward = 0

    actor_weight = 0
    critic_weight = 0


    # actor_prev = copy.deepcopy(actor)
    # critic_prev = copy.deepcopy(critic)

    for step in range(1,STEPS_PER_EPISODE+1):

        # Select action

        state_tensor = torch.FloatTensor(state).to(device)

        action_probs= actor(state_tensor)

        action = torch.multinomial(action_probs, 1).item()  # Sample action

        ###################

        uav_loc = np.reshape(state[:-QUAD.user_number], (2,-1))
        user_profile = state[-QUAD.user_number:]

        scheduling = schedule(user_profile,QUAD.user_loc,uav_loc,QUAD.transmit_pow,QUAD.noise,QUAD.height)

        next_state = QUAD.next_state(action,uav_loc,user_profile,scheduling)

        uav_loc = np.reshape(next_state[:-QUAD.user_number], (2,-1))


        reward , done = QUAD.my_reward(uav_loc, scheduling, user_profile)

        state_tensor = torch.tensor(np.array(state), dtype=torch.float32, device=device)
        action = torch.tensor(action, dtype=torch.int64, device=device)
        reward_tensor = torch.tensor(reward, dtype=torch.float32, device=device)
        next_tensor = torch.tensor(np.array(next_state), dtype=torch.float32, device=device)


################################ training

        # Compute Advantage
        value = critic(state_tensor)
        next_value = critic(next_tensor).detach()
        advantage = reward_tensor + (0.99 * next_value * (not done)) - value

        # Update Actor
        log_prob = torch.log(action_probs[action])
        loss_actor = -log_prob * advantage.detach()
        actor_optimizer.zero_grad()
        loss_actor.backward()
        actor_optimizer.step()

        # Update Critic
        loss_critic = advantage.pow(2)
        critic_optimizer.zero_grad()
        loss_critic.backward()
        critic_optimizer.step()


################################ end of training


        # Move to next state
        state = next_state

        total_reward += reward


        if done:
            break


    reward_acul = np.append(reward_acul,total_reward)

    if episode%10 == 0 :
        toc = time.time()
        print(f"episode {episode}/{EPISODES}, mean past 10 Reward: {np.mean(reward_acul[-10:])},actor weights: {weight_change(actor,actor_prev)}, critic weights: {weight_change(critic,critic_prev)}, elapsed: {(toc-tic)/60} minutes")
        actor_prev.load_state_dict(actor.state_dict())
        critic_prev.load_state_dict(critic.state_dict())

    if episode%100 == 0:

        check_point = {
        'actor_state':actor.state_dict(),
        'actor_optimizer': actor_optimizer.state_dict(),
        'critic_state':critic.state_dict(),
        'critic_optimizer': critic_optimizer.state_dict(),
        'episode' : episode
        }

        torch.save(check_point,"model.pth")
        np.save("reward.npy",reward_acul)
        np.save(f"episode.npy",episode)



episode 10/10000, mean past 10 Reward: -406.10477025462495,actor weights: 210.94754314422607, critic weights: 19237.10738171637, elapsed: 0.16677460670471192 minutes
episode 20/10000, mean past 10 Reward: -399.5825826678112,actor weights: 192.43238079547882, critic weights: 15818.251704357564, elapsed: 0.3227917551994324 minutes
episode 30/10000, mean past 10 Reward: 598.3431316479354,actor weights: 38.648295886814594, critic weights: 5551.548679187894, elapsed: 0.6767906149228414 minutes
episode 40/10000, mean past 10 Reward: 278.4165853431806,actor weights: 46.733669489622116, critic weights: 15391.30018159002, elapsed: 0.998733667532603 minutes
episode 50/10000, mean past 10 Reward: 417.21556837131004,actor weights: 1.8870290666818619, critic weights: 11787.791447222233, elapsed: 1.3456382036209107 minutes
episode 60/10000, mean past 10 Reward: 242.41206445997477,actor weights: 65.66957950592041, critic weights: 10684.325760006905, elapsed: 1.6496082027753194 minutes
episode 70/1000

KeyboardInterrupt: 